In [1]:
!pip install -q transformers datasets evaluate spacy
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but yo

In [2]:
# 1. Import thư viện
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import numpy as np
import evaluate
import spacy

2025-06-05 09:12:46.724946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749114767.148723      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749114767.275904      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# 2. Đọc dữ liệu và tiền xử lý
df = pd.read_csv('/kaggle/input/imdb-train/imdb_aspect_sentiment_train.csv')

# Giữ nhãn positive, negative và map label thành 1/0
df = df[df["sentiment"].isin(["positive", "negative"])].copy()
df["label"] = df["sentiment"].map({"positive": 1, "negative": 0})

# # Tạo cột input_text dạng "aspect: sentence"
df["input_text"] = df.apply(lambda row: f"{row['aspect']}: {row['sentence']}", axis=1)

In [4]:
# 3. Tạo Huggingface Dataset, loại bỏ cột không cần thiết
dataset = Dataset.from_pandas(df)
dataset = dataset.remove_columns(["sentence", "aspect", "sentiment"])

# 4. Chia train/test
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

print("Số dòng train sau chia:", len(train_dataset))
print("Số dòng validation sau chia:", len(val_dataset))

Số dòng train sau chia: 7867
Số dòng validation sau chia: 1967


In [5]:
# 5. Load tokenizer và model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# 6. Hàm preprocess tokenizing
def preprocess_function(examples):
    return tokenizer(
        examples['input_text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
print("Số dòng tokenized_train:", len(tokenized_train))
print("Số dòng tokenized_val:", len(tokenized_val))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7867 [00:00<?, ? examples/s]

Map:   0%|          | 0/1967 [00:00<?, ? examples/s]

Số dòng tokenized_train: 7867
Số dòng tokenized_val: 1967


In [6]:
# 7. Đổi tên cột label thành labels (yêu cầu Trainer)
if "label" in tokenized_train.column_names:
    tokenized_train = tokenized_train.rename_column("label", "labels")
    tokenized_val = tokenized_val.rename_column("label", "labels")

# 8. Đặt định dạng torch
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# 9. DatasetDict
encoded_dataset = DatasetDict({
    "train": tokenized_train,
    "validation": tokenized_val
})

# 10. Metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_metric.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1_metric.compute(predictions=preds, references=labels)["f1"],
    }

# 11. TrainingArguments và Trainer
training_args = TrainingArguments(
    output_dir="/kaggle/working/bert_result",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=None,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 12. Huấn luyện model
trainer.train()

/tmp/ipykernel_19/403793896.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.469100,0.460576,0.781901,0.742342
2,0.416700,0.456258,0.801220,0.806340
3,0.356000,0.405334,0.814947,0.804721
4,0.309000,0.435604,0.813930,0.810950
5,0.280700,0.472755,0.813421,0.801299


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=2460, training_loss=0.37026361445101297, metrics={'train_runtime': 780.9587, 'train_samples_per_second': 50.368, 'train_steps_per_second': 3.15, 'total_flos': 2587368340646400.0, 'train_loss': 0.37026361445101297, 'epoch': 5.0})

In [7]:
# 2. Đọc dữ liệu và tiền xử lý
df1 = pd.read_csv('/kaggle/input/imdb-test/imdb_aspect_sentiment_test.csv')

# Giữ nhãn positive, negative và map label thành 1/0
df1 = df1[df1["sentiment"].isin(["positive", "negative"])].copy()
df1["label"] = df1["sentiment"].map({"positive": 1, "negative": 0})

# # Tạo cột input_text dạng "aspect: sentence"
df1["input_text"] = df1.apply(lambda row: f"{row['aspect']}: {row['sentence']}", axis=1)
# Chuyển sang HuggingFace Dataset
dataset = Dataset.from_pandas(df1[["input_text", "label"]])

In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["input_text"], truncation=True, padding="max_length", max_length=128)

# Áp dụng tokenizer
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/1824 [00:00<?, ? examples/s]

In [9]:
model = BertForSequenceClassification.from_pretrained("/kaggle/working/bert_result/checkpoint-1968")

In [10]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"],
    }
eval_args = TrainingArguments(
    output_dir="/kaggle/working/test_output",
    per_device_eval_batch_size=8,
    report_to="none",
)
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=eval_args,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=tokenized_dataset)
from pprint import pprint

pprint(metrics)

/tmp/ipykernel_19/1433912918.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_accuracy': 0.6233552631578947,
 'eval_f1': 0.6393700787401574,
 'eval_loss': 1.0030860900878906,
 'eval_runtime': 9.7233,
 'eval_samples_per_second': 187.591,
 'eval_steps_per_second': 11.724}


In [11]:
# 1. Load dữ liệu từ CSV
df = pd.read_csv("/kaggle/input/semeval-restaurant/Restaurants_Train_v2.csv")

# 2. In số dòng theo từng nhãn
print("Tổng số dòng:", len(df))
print("Số dòng có nhãn 'neutral':", (df["polarity"] == "neutral").sum())
print("Số dòng có nhãn 'positive':", (df["polarity"] == "positive").sum())
print("Số dòng có nhãn 'negative':", (df["polarity"] == "negative").sum())
print("Các nhãn khác:", df["polarity"].value_counts())


Tổng số dòng: 3693
Số dòng có nhãn 'neutral': 633
Số dòng có nhãn 'positive': 2164
Số dòng có nhãn 'negative': 805
Các nhãn khác: polarity
positive    2164
negative     805
neutral      633
conflict      91
Name: count, dtype: int64


In [12]:
# Chỉ giữ lại các nhãn cần thiết
df = df[df["polarity"].isin(["positive", "negative"])].copy()

print("Số dòng có nhãn 'positive':", (df["polarity"] == "positive").sum())
print("Số dòng có nhãn 'negative':", (df["polarity"] == "negative").sum())

# Map nhãn thành 0 và 1
df["label"] = df["polarity"].map({"positive": 1, "negative": 0})

# Tạo input_text theo định dạng "aspect: sentence"
df["input_text"] = df.apply(lambda row: f"{row['Aspect Term']}: {row['Sentence']}", axis=1)

# Chuyển sang HuggingFace Dataset
dataset = Dataset.from_pandas(df[["input_text", "label"]])

Số dòng có nhãn 'positive': 2164
Số dòng có nhãn 'negative': 805


In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["input_text"], truncation=True, padding="max_length", max_length=128)

# Áp dụng tokenizer
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/2969 [00:00<?, ? examples/s]

In [14]:
model = BertForSequenceClassification.from_pretrained("/kaggle/working/bert_result/checkpoint-1968")

In [15]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"],
    }
eval_args = TrainingArguments(
    output_dir="/kaggle/working/test_output1",
    per_device_eval_batch_size=8,
    report_to="none",
)
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=eval_args,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=tokenized_dataset)
from pprint import pprint

pprint(metrics)

/tmp/ipykernel_19/2651515382.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_accuracy': 0.7696194004715392,
 'eval_f1': 0.8455981941309256,
 'eval_loss': 0.5180588960647583,
 'eval_runtime': 16.0958,
 'eval_samples_per_second': 184.458,
 'eval_steps_per_second': 11.556}


In [16]:
model = BertForSequenceClassification.from_pretrained("/kaggle/working/bert_result/checkpoint-1476")
# 13. Hàm phân loại sentiment cho câu + khía cạnh mới
import re
import spacy

# Load mô hình spaCy tiếng Anh
nlp = spacy.load("en_core_web_sm")

def extract_aspects(sentence):
    doc = nlp(sentence)
    aspects = list(set([token.text.lower() for token in doc if token.pos_ in {"NOUN", "PROPN"}]))
    results = []
    for asp in aspects:
        sentiment = classify_sentiment_bert(asp, sentence)
        results.append((asp, sentiment))
    return results

def classify_sentiment_bert(aspect, clause):
    input_text = f"{aspect} [SEP] {clause}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        pred = outputs.logits.argmax(dim=-1).item()
    return "positive" if pred == 1 else "negative"

In [17]:
#Test
#The actor was bad.
#Stunning visual effects, bad actor
#The visual effects were stunning but the plot was boring
#The visual effects and the plot was boring
#The plot, the visual effects and the actor were bad
#The visual effects, the actor and the plot were bad
#The visual effects were stunning, the plot was boring
# Ví dụ test
sentences = [
    "The plot was terrible",
    "The visual effects were bad but the acting was amazing",
    "The visual effects and the plot was boring",
    "The scene, the visual effects and the actor were good",
    "The visual effects, the acting and the plot were bad",
    "The visual effects were boring, the plot was good",
    "The visual effects, character and music dissapointed me"
]

for sent in sentences:
    print(f"\nSentence: {sent}")
    for aspect, sentiment in extract_aspects(sent):
        print(f"Aspect: {aspect} → Sentiment: {sentiment}")


Sentence: The plot was terrible
Aspect: plot → Sentiment: negative

Sentence: The visual effects were bad but the acting was amazing
Aspect: effects → Sentiment: negative
Aspect: acting → Sentiment: positive

Sentence: The visual effects and the plot was boring
Aspect: plot → Sentiment: negative
Aspect: effects → Sentiment: negative

Sentence: The scene, the visual effects and the actor were good
Aspect: actor → Sentiment: positive
Aspect: effects → Sentiment: positive
Aspect: scene → Sentiment: positive

Sentence: The visual effects, the acting and the plot were bad
Aspect: plot → Sentiment: negative
Aspect: effects → Sentiment: negative
Aspect: acting → Sentiment: negative

Sentence: The visual effects were boring, the plot was good
Aspect: plot → Sentiment: negative
Aspect: effects → Sentiment: negative

Sentence: The visual effects, character and music dissapointed me
Aspect: character → Sentiment: negative
Aspect: effects → Sentiment: negative
Aspect: music → Sentiment: negative


In [18]:
# # 13. Hàm phân loại sentiment cho câu + khía cạnh mới
# import re
# import spacy

# # Load mô hình spaCy tiếng Anh
# nlp = spacy.load("en_core_web_sm")

# def extract_aspects(sentence):
#     doc = nlp(sentence)
#     extracted = [token.text.lower() for token in doc if token.pos_ in {"NOUN", "PROPN"}]
#     return list(set(extracted))  # loại trùng

# def split_clauses(sentence):
#     # Tách mệnh đề theo dấu câu và liên từ phổ biến
#     return [cl.strip() for cl in re.split(r"[.,;]| but | although | however | and ", sentence.lower()) if cl.strip()]

# def detect_listed_aspects(sentence, keywords):
#     sentence_lower = sentence.lower()
#     found = [kw for kw in keywords if kw in sentence_lower]
#     if len(found) < 2:
#         return False
#     be_verbs = [" is ", " was ", " are ", " were "]
#     for bv in be_verbs:
#         pos_bv = sentence_lower.find(bv)
#         if pos_bv != -1:
#             # Kiểm tra các khía cạnh đứng liền nhau trước be verb
#             positions = sorted([sentence_lower.find(k) for k in found if sentence_lower.find(k) < pos_bv])
#             for i in range(len(positions) - 1):
#                 if positions[i+1] - positions[i] < 25:  # khoảng cách ký tự nhỏ
#                     return True
#     return False

# def map_aspect_to_clause(aspects, sentence):
#     clauses = split_clauses(sentence)
#     mapped = []
#     if detect_listed_aspects(sentence, aspects):
#         # Trường hợp liệt kê nhiều khía cạnh cùng một cảm xúc
#         for asp in aspects:
#             mapped.append((asp, sentence.lower()))
#     else:
#         # Ánh xạ từng khía cạnh vào mệnh đề chứa nó (nếu có)
#         for asp in aspects:
#             found = next((c for c in clauses if asp in c), None)
#             mapped.append((asp, found if found else sentence.lower()))
#     return mapped

# def classify_sentiment_bert(aspect, clause):
#     input_text = f"{aspect}: {clause}"
#     inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)
#     inputs = {k: v.to(model.device) for k, v in inputs.items()}
#     model.eval()
#     with torch.no_grad():
#         outputs = model(**inputs)
#         pred = outputs.logits.argmax(dim=-1).item()
#     return "positive" if pred == 1 else "negative"